In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import re
import pandas as pd
import os
import time ## give a delay execution
import warnings
import numpy as np
from urllib.request import urlretrieve
import base64

from PIL import Image
from io import BytesIO

In [17]:
url = "https://www.whoscored.com/Regions/74/Tournaments/22/France-Ligue-1"
driver = webdriver.Firefox(executable_path='../geckodriver') # define the webdriver (Firefox, PhantomJS, chrome,...)
driver.implicitly_wait(30)  # sticky timeout to wait implicitly wait for an element to be found
driver.get(url) # load the webpage
tab = driver.find_element_by_id("tournament-fixture")
link = tab.find_elements_by_class_name('result')[:4]
links = [l.find_element_by_tag_name('a').get_attribute("href") for l in link]
driver.quit()

## Collect the heatmap of the match

In [96]:
url = "https://www.whoscored.com/Matches/1376617/Live/France-Ligue-1-2019-2020-Bordeaux-Nimes"
driver = webdriver.Firefox(executable_path='../geckodriver') # define the webdriver (Firefox, PhantomJS, chrome,...)
driver.implicitly_wait(30)  # sticky timeout to wait implicitly wait for an element to be found
driver.get(url) # load the webpage
time.sleep(5)
try:
    driver.find_elements_by_class_name("qc-cmp-button")[1].click()
except:
    pass
ul = driver.find_element_by_id("live-match-options")
ul.find_elements_by_tag_name('li')[3].click()
heatmap = driver.find_element_by_id("heatmap-pitches")
heatmaps = heatmap.find_elements_by_class_name("canvas-container")
heatmaps = [h.find_element_by_tag_name('canvas') for h in heatmaps]
for i, h in enumerate(heatmaps):
    location = h.location
    size = h.size
    driver.execute_script("window.scrollTo(0, {})".format(location['y'])) 
    png = driver.get_screenshot_as_png() # saves screenshot of entire page
    time.sleep(5)
    im = Image.open(BytesIO(png)) # uses PIL library to open image in memory
    left, upper, right, lower = (location['x'], 0, location['x'] + size['width'], size['height'])
    im = im.crop((left, upper, right, lower)) # defines crop points
    im.save('heatmaps/heat_{}.png'.format(i)) # saves new cropped image
    
driver.quit()